In [1]:
import math, sys, os

sys.path.insert(0, os.path.join(os.getcwd(), 'src'))
from plotter import *
from kalman import *


# Import tdmclient Notebook environment:
import tdmclient.notebook
await tdmclient.notebook.start()

In [2]:
%%run_python

state = 1

@onevent
def move(speed_left, speed_right):
    global motor_left_target, motor_right_target, state
    if state == 1 :
        motor_left_target = speed_left
        motor_right_target = speed_right
    
@onevent 
def button_center():
    global state, motor_left_target, motor_right_target
    state = 0
    motor_left_target = 0
    motor_right_target = 0

In [3]:
def send_command():
    if optimal_command[step][0] == optimal_command[step][1] and optimal_command[step][0] != 0:
        send_event("move", optimal_command[step][0] + 2, optimal_command[step][1] - 2)
    else :
        send_event("move", optimal_command[step][0], optimal_command[step][1])

In [4]:
@tdmclient.notebook.sync_var
def get_speed():
    return motor_left_speed, motor_right_speed

def kalman_call():
    global step, x_est, start, rt
    
    if step < len(optimal_command) and camera_on != True:
        #retrieves the equivalent of the command that was applied Ts seconds ago
        speed_left = optimal_command[step][0]
        speed_right = optimal_command[step][1]

        speed_trans = (speed_left + speed_right) * thymio_speed_to_mms / 2
        speed_rot =  (speed_right - speed_left) * (thymio_speed_to_mms / distance_wheel)

        theta = x_est[-1][2];

        x_est[-1][3] = speed_trans * math.cos(theta)
        x_est[-1][4] = speed_trans * math.sin(theta)
        x_est[-1][5] = speed_rot

        #gets the right matrix corresponding to the command :
        #Thymio goes forward
        if speed_left == speed_right :
            Q = Q_F
            R = R_F
            A = A_F

        #Thymio is rotating
        else :
            Q = Q_R
            R = R_R
            A = A_R


        #measures the speed of the robot
        meas_speed_left,  meas_speed_right = get_speed()

        #computes the estimated states
        new_x_est, new_P_est = kalman_filter(meas_speed_left, meas_speed_right, x_est[-1], P_est[-1], Q, R, A)

        x_est.append(new_x_est)
        P_est.append(new_P_est)

        step += 1

        if step < len(optimal_command) :
            send_command()
    else :
        rt.stop()

In [5]:
camera_on = False

#initial state :
x0 = 10
y0 = 10
theta0 = 0
x_dot0 = 0 
y_dot0 = 0
theta_dot0 = 0 

x_est = [np.array([[x0], [y0], [theta0], [x_dot0], [y_dot0], [theta_dot0]])]

if x_dot0 == y_dot0 :
    P_est = [np.diag(np.array([0.001, 0.001, 0.000001, q_F_x_dot, q_F_y_dot, q_F_theta_dot]))]

else :
    P_est = [np.diago(np.array([0.001, 0.001, 0.000001, q_R_x_dot, q_R_y_dot, q_R_theta_dot]))]

step = 0
optimal_command = [[-100, 100], [-100, 100], [-100, 100], [-100, 100], [-100, 100], [-100, 100], [-100, 100], [-100, 100], [-100, 100], [-100, 100], [-100, 100], [-100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [100, 100], [0, 0]]

#start of Kalman
send_command()

rt = RepeatedTimer(Ts, kalman_call)